In [ ]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings

from langchain.prompts import PromptTemplate
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import (
    FewShotPromptTemplate
)
import pickle
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
def get_embedding():
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-small',
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
        )
    return embeddings

def make_dict(dir='train.csv'):
    df = pd.read_csv(dir)
    df.drop('SAMPLE_ID', axis=1, inplace=True)
    
    return df.to_dict(orient='records')



def get_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-small',
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True}
        )
    return embeddings

def load_and_vectorize(csv_path, db_path):
    if os.path.exists(db_path) and os.path.exists(db_path + '_metadata.pkl'):
        print("Loading FAISS DB from:", db_path)
        db, metadata = load_faiss_db(db_path)
        return db

    # CSV 파일을 불러와 데이터프레임 생성
    train_df = pd.read_csv(csv_path)
    train_df.drop('SAMPLE_ID', axis=1, inplace=True)
    trainset = train_df.to_dict(orient='records')
    print("Loaded Fewshot Set:", trainset[:3])
    
    # 벡터화할 텍스트 생성
    to_vectorize = ["\n\n".join(example.values()) for example in trainset]
    
    # 벡터화 및 FAISS DB 생성
    fewshow_vectordb = FAISS.from_texts(to_vectorize, embedding=get_embedding(), metadatas=trainset)
    
    # FAISS DB 저장
    save_faiss_db(fewshow_vectordb, db_path)
    
    return fewshow_vectordb

def save_faiss_db(db, db_path):
    db.save_local(db_path)
    with open(db_path + '_metadata.pkl', 'wb') as f:
        pickle.dump(db, f)

def load_faiss_db(db_path):
    db = FAISS.load_local(db_path, embeddings=get_embedding(), allow_dangerous_deserialization=True)
    with open(db_path + '_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    return db, metadata

def load_chunks_make_docdb(pdf_directory, db_path):
    if os.path.exists(db_path) and os.path.exists(db_path + '_metadata.pkl'):
        print("Loading FAISS DB from:", db_path)
        db, metadata = load_faiss_db(db_path)
        return db

    print("Loading PDF files from:", pdf_directory)
    documents = []

    # PDF 파일들을 로드하여 분할
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf').replace('\\', '/'))
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        documents.extend(pdf_documents)
    
    # 분할된 텍스트를 벡터로 변환하여 FAISS DB에 저장
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = chunk_splitter.split_documents(documents)
    print("Done.", len(chunks), "chunks")
    
    print("Creating FAISS DB")
    # FAISS DB 생성 및 저장
    db = FAISS.from_documents(chunks, embedding=get_embedding())
    save_faiss_db(db, db_path)
    print("Done.")
    
    return db

def setup_llm_pipeline(model_id):

    model_id = model_id

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        return_full_text=False,
        max_new_tokens=128,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

def make_fewshot_prompt(fewshot_vectordb, k = 3):
    # Semantic Similarity Example Selector 설정
    example_prompt = PromptTemplate.from_template("Question: {Question}\nAnswer: {Answer}\nSource: {Source}")

    example_selector = SemanticSimilarityExampleSelector(
        vectorstore=fewshot_vectordb,
        k=k,
    )

    # FewShotPromptTemplate 생성
    fewshot_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        suffix="Question: {input}",
        input_variables=["input"],
    )
    return fewshot_prompt

def make_fewshot_string(fewshot_prompt, train_retriever, buff):
    ex_qa = fewshot_prompt.invoke({"input": buff['Question']}).to_string()
    fewshot_list = ex_qa.split('\n\n')[:-1]
    for i, entry in enumerate(fewshot_list):
        question = entry.split('\n')[0]
        question = question.replace('Question: ', '')
        retrieved_docs = train_retriever.invoke(question)
        entry += retrieved_docs[0].page_content
        num = "Example {}\n".format(i+1)
        fewshot_list[i] = num + entry + '\n\n'
    return str(fewshot_list)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
import transformers
def setup_llm_pipeline(model_id):
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
        max_new_tokens=256,
    )

    hf = HuggingFacePipeline(pipeline=pipeline)
    return hf


In [ ]:
fewshot_db = load_and_vectorize('train.csv', './fewshot_faiss_db')
fewshot_prompt = make_fewshot_prompt(fewshot_db)

train_db = load_chunks_make_docdb('./train_source', './train_faiss_db')
train_retriever = train_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})

test_db = load_chunks_make_docdb('./test_source', './test_faiss_db')
test_retriever = test_db.as_retriever(search_type = "mmr",search_kwargs={'k': 1, 'fetch_k': 50})

train_dict = make_dict('train.csv')
test_dict = make_dict('test.csv')

Loading FAISS DB from: ./fewshot_faiss_db
Loading FAISS DB from: ./train_faiss_db
Loading FAISS DB from: ./test_faiss_db


In [ ]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

llm = setup_llm_pipeline(model_id)

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\우승우\.cache\huggingface\hub\models--meta-llama--Meta-Llama-3.1-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]
Some parameter

In [ ]:

i = 1
fewshot_str = make_fewshot_string(fewshot_prompt, train_retriever, test_dict[i])
        # print(fewshot_str)
full_template = """
너는 사람들에게 방대한 재정 데이터를 일반 국민과 전문가들이 이해할 수 있는 방식으로 전달하고 싶어한다.
너는 이를 위한 몇가지 예시 참고해서  재정 보고서, 예산 설명자료, 기획재정부 보도자료 등 다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 응답을 제시해줘.
반드시 context를 참고하여 답변을 작성해야해.
예시를 참고하여 아래 질문에 대한 답변을 작성해주세요.

"""+f"""
이건 질문과 관련된 예시야.

{fewshot_str}
"""+"""
        
이 context를 참고하여 아래 질문에 대한 답변을 작성해주세요.
{context}

이제 진짜로 한번 해봐.
질문: {input}

"""
print(full_template)


너는 사람들에게 방대한 재정 데이터를 일반 국민과 전문가들이 이해할 수 있는 방식으로 전달하고 싶어한다.
너는 이를 위한 몇가지 예시 참고해서  재정 보고서, 예산 설명자료, 기획재정부 보도자료 등 다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 응답을 제시해줘.
반드시 context를 참고하여 답변을 작성해야해.
예시를 참고하여 아래 질문에 대한 답변을 작성해주세요.


이건 질문과 관련된 예시야.

Question: 창업사업화지원의 사업목적은 무엇인가?
Answer: 창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.
Source: 중소벤처기업부_창업사업화지원
Context: 중소벤처기업부_창업사업화지원

./train_source/중소벤처기업부_창업사업화지원.pdf

창업사업화지원의 사업목적은 무엇인가?

창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.

Question: 언제 청년 일자리 대책의 일환으로 기술혁신 기반 예비창업자의 사업화를 지원하는 '기술혁신형 창업기업 지원사업'이 신설되었나요?
Answer: 기술혁신형 창업기업 지원사업(현, 예비창업패키지)'은 '18년에 신설되었습니다.
Source: 중소벤처기업부_창업사업화지원
Context: 중소벤처기업부_창업사업화지원

./train_source/중소벤처기업부_창업사업화지원.pdf

언제 청년 일자리 대책의 일환으로 기술혁신 기반 예비창업자의 사업화를 지원하는 '기술혁신형 창업기업 지원사업'이 신설되었나요?

기술혁신형 창업기업 지원사업(현, 예비창업패키지)'은 '18년에 신설되었습니다.

Question: 2024년 중소벤처기업부의 창업사업화지원 사업에서 어떤 기관들이 사업시행주체로 참여하는가?
Answer: 중소벤처기업부, 창업진

In [ ]:
prompt = PromptTemplate.from_template(full_template)
qa_chain = (
        {
            "context": test_retriever | format_docs,
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm.bind(skip_prompt=True)
        | StrOutputParser()
        )

answer = qa_chain.invoke(test_dict[i]['Question'])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(answer)   


너는 사람들에게 방대한 재정 데이터를 일반 국민과 전문가들이 이해할 수 있는 방식으로 전달하고 싶어한다.
너는 이를 위한 몇가지 예시 참고해서  재정 보고서, 예산 설명자료, 기획재정부 보도자료 등 다양한 재정 관련 텍스트 데이터를 활용해 주어진 질문에 대해 정확도가 높은 응답을 제시해줘.
반드시 context를 참고하여 답변을 작성해야해.
예시를 참고하여 아래 질문에 대한 답변을 작성해주세요.


이건 질문과 관련된 예시야.

Question: 창업사업화지원의 사업목적은 무엇인가?
Answer: 창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.
Source: 중소벤처기업부_창업사업화지원
Context: 중소벤처기업부_창업사업화지원

./train_source/중소벤처기업부_창업사업화지원.pdf

창업사업화지원의 사업목적은 무엇인가?

창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.

Question: 언제 청년 일자리 대책의 일환으로 기술혁신 기반 예비창업자의 사업화를 지원하는 '기술혁신형 창업기업 지원사업'이 신설되었나요?
Answer: 기술혁신형 창업기업 지원사업(현, 예비창업패키지)'은 '18년에 신설되었습니다.
Source: 중소벤처기업부_창업사업화지원
Context: 중소벤처기업부_창업사업화지원

./train_source/중소벤처기업부_창업사업화지원.pdf

언제 청년 일자리 대책의 일환으로 기술혁신 기반 예비창업자의 사업화를 지원하는 '기술혁신형 창업기업 지원사업'이 신설되었나요?

기술혁신형 창업기업 지원사업(현, 예비창업패키지)'은 '18년에 신설되었습니다.

Question: 2024년 중소벤처기업부의 창업사업화지원 사업에서 어떤 기관들이 사업시행주체로 참여하는가?
Answer: 중소벤처기업부, 창업진